In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
# This bit of code allows me to import modules for adjacent folders:

import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
lobbyists_df = pd.read_excel('../Lobbyist Export.xls', 'Lobbyists')
lobbyists_df.columns = lobbyists_df.columns.str.replace(' ', '_')
lobbyists_df.columns = lobbyists_df.columns.str.lower()
lobbyists_df.abn = pd.to_numeric(lobbyists_df.abn.str.replace(' ', ''))
lobbyists_df

,business_name,trading_name,abn,web_address,address,suburb,state,post_code,name_of_owner(s)
0,Advanced Outcomes,Advanced Outcomes,59719341477,NaN,Redacted,Redacted,SA,5006,Robert Underdown
1,Alistair Nicholas Consulting Pty Ltd,Alistair Nicholas Consulting Pty Ltd,81655939852,https://www.alistairnicholas.com/,"Suite 2301, Forum West, 3 Herbert Street",St Leonards,NSW,2065,Alistair John Nicholas
2,Alkar Pty Ltd,Allan King Consultancy,16007797793,NaN,PO Box 4144,MANUKA,ACT,2603,Allan King\nKaren King
3,Anacta Strategies Pty Ltd,Anacta Strategies,64633978677,Anacta.com.au,GPO Box 1349,Brisbane,QLD,4000,Melissa Thomas
4,PolicyWonks,Anderson Solutions Trust,50463070316,NaN,42 Balowrie Street\nHamilton QLD 4007\nAustralia,Hamilton,QLD,4007,Kirby Anderson Dianne Anderson
...,...,...,...,...,...,...,...,...,...
95,Wells Haslem Mayhew Strategic Public Affairs P...,Wells Haslem Mayhew Strategic Public Affairs P...,52159456685,whmspa.com.au,"Level 2, 65 Walker Street",NORTH SYDNEY,NSW,2060,Philip John Wells\nBenjamin Haslem\nAlexandra ...
96,Wendy Campana Consulting Pty Ltd,Wendy Campana Consulting Pty Ltd,53649896775,NaN,Redacted,Redacted,SA,0,Wendy Campana
97,Wilkinson Butler Pty Ltd,Wilkinson Butler,95164204111,www.wilkinsonbutler.com,Level 26\n1-7 Bligh Street\n,SYDNEY,NSW,2000,Andrew Butler\nPeter Wilkinson
98,Willard Public Affairs Pty Limited,Willard Public Affairs,17165851856,www.willard.com.au,"Level 13, 84 Pitt Street",SYDNEY,NSW,2000,Willard Consulting Pty Ltd ATF David Miles Fam...


In [12]:
employees_df = pd.read_excel('../Lobbyist Export.xls', 'Employees')
employees_df.columns = employees_df.columns.str.replace(' ', '_')
employees_df.columns = employees_df.columns.str.lower()
employees_df.lobbyist_abn = pd.to_numeric(employees_df.lobbyist_abn.str.replace(' ', ''))
employees_df

,lobbyist_abn,lobbyist_business_name,lobbyist_trading_name,employee_type,employee_name,employee_position,employee_restriction,employee_start_date,employee_end_date
0,59719341477,Advanced Outcomes,Advanced Outcomes,Persons undertaking lobbying activities,Robert Underdown,Director,None,02/10/2018,NaN
1,81655939852,Alistair Nicholas Consulting Pty Ltd,Alistair Nicholas Consulting Pty Ltd,Persons undertaking lobbying activities,Alistair Nicholas,Director,None,06/07/2021,NaN
2,16007797793,Alkar Pty Ltd,Allan King Consultancy,Persons undertaking lobbying activities,Allan King,Director,None,06/04/2017,NaN
3,64633978677,Anacta Strategies Pty Ltd,Anacta Strategies,Persons undertaking lobbying activities,Evan Moorhead,Director,None,26/04/2022,NaN
4,64633978677,Anacta Strategies Pty Ltd,Anacta Strategies,Persons undertaking lobbying activities,Liam O'Regan,Senior Associate,None,26/04/2022,NaN
...,...,...,...,...,...,...,...,...,...
480,17165851856,Willard Public Affairs Pty Limited,Willard Public Affairs,Persons undertaking lobbying activities,Matthew Moran,Principal,None,23/06/2016,NaN
481,17165851856,Willard Public Affairs Pty Limited,Willard Public Affairs,Persons undertaking lobbying activities,Andrew Plumley,Director,None,23/06/2016,NaN
482,17165851856,Willard Public Affairs Pty Limited,Willard Public Affairs,Persons undertaking lobbying activities,Mary Andrew,Consultant,None,23/06/2016,13/07/2018
483,17165851856,Willard Public Affairs Pty Limited,Willard Public Affairs,Persons undertaking lobbying activities,Rosemary Apps,Office Manager,None,23/06/2016,09/01/2018


In [13]:
clients_df = pd.read_excel('../Lobbyist Export.xls', 'Clients')
clients_df.columns = clients_df.columns.str.replace(' ', '_')
clients_df.columns = clients_df.columns.str.lower()
clients_df.lobbyist_abn = pd.to_numeric(clients_df.lobbyist_abn.str.replace(' ', ''))
clients_df

,lobbyist_abn,lobbyist_business_name,lobbyist_trading_name,client_name,client_start_date,client_end_date
0,59719341477,Advanced Outcomes,Advanced Outcomes,Commercial & General Group of Companies,02/10/2018,NaN
1,59719341477,Advanced Outcomes,Advanced Outcomes,Backpacks 4 SA Kids,22/11/2018,NaN
2,59719341477,Advanced Outcomes,Advanced Outcomes,Cooperative Research Centre for Contamination ...,22/11/2018,NaN
3,59719341477,Advanced Outcomes,Advanced Outcomes,Employers Mutual Limited (EML),22/11/2018,01/05/2019
4,59719341477,Advanced Outcomes,Advanced Outcomes,Argonaut Resources NL,15/03/2019,NaN
...,...,...,...,...,...,...
1052,62613694189,Woodhill Communication Pty Ltd,Woodhill Communication,Sirtex Technologies Pty Ltd,17/05/2016,29/04/2018
1053,62613694189,Woodhill Communication Pty Ltd,Woodhill Communication,Society of Interventional Radiologists,01/11/2016,31/12/2018
1054,62613694189,Woodhill Communication Pty Ltd,Woodhill Communication,Enware Pty Ltd,17/05/2016,29/04/2018
1055,62613694189,Woodhill Communication Pty Ltd,Woodhill Communication,Group of 100 inc (G100),29/04/2018,NaN


In [5]:
#!pip install selenium
# !pip install --upgrade pip
# !pip install sqlalchemy
# !pip install sqlalchemy_utils
# !pip install regex
# !pip install xlrd
# !ls ../

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 511.8 kB/s eta 0:00:00a 0:00:01


In [3]:
import sqlite3
cnx = sqlite3.connect('../data.db')
pd.read_sql_query("select * from lobbyist_nsw", cnx)

,id,name,abn,trading_name,on_watch_list,status,status_note,last_updated
0,1,Alistair Nicholas Consulting Pty Ltd,81655939852,Alistair Nicholas Consulting Pty Ltd,,Active,Status Note: Authorised to communicate and mee...,Details Last Updated: 10 Jan 2022 / 12:00
1,2,Alkar Pty Ltd,16007797793,Allan King Consultancy,,Active,Status Note: Authorised to communicate and mee...,Details Last Updated: 08 Dec 2016 / 12:00
2,3,Alpha Consult Pty Ltd,39612003460,Alpha Consult,,Active,Status Note: Authorised to communicate and mee...,Details Last Updated: 21 Jan 2022 / 12:00
3,4,ATLAS ALLIANCE PTY LTD,46606084913,Atlas Alliance,,Active,Status Note: Authorised to communicate and mee...,Details Last Updated: 02 Jun 2021 / 12:00
4,5,Australian Public Affairs Limited Partnership,20098705403,Australian Public Affairs,,Active,Status Note: Authorised to communicate and mee...,Details Last Updated: 25 Jul 2022 / 12:00
...,...,...,...,...,...,...,...,...
340,341,The Trustee for POWER FAMILY TRUST,64964952044,Govstrat,,Cancelled,Status Note: Registration cancelled on 31/10/2019,Details Last Updated: 31 Oct 2019 / 12:00
341,342,TransAdvantage Pty Ltd,37166803621,TransAdvantage,,Cancelled,Status Note: Registration cancelled on 20/12/2017,Details Last Updated: 18 Jun 2019 / 12:00
342,343,WABSLEY PTY LIMITED,27166543193,Watt Advocacy & Communications,,Cancelled,Status Note: Registration cancelled on 11/7/2017,Details Last Updated: 11 Jul 2017 / 12:00
343,344,Woodhill Communication Pty Ltd,62613694189,Woodhill Communication Pty Ltd,,Cancelled,Status Note: Registration cancelled on 11/7/2017,Details Last Updated: 08 Aug 2018 / 12:00


In [4]:
pd.read_sql_query("select * from lobbyist_nsw_employee", cnx)

,id,lobbyist_nsw_id,postion,name,active,date_added
0,1,None,Director,Alistair Nicholas,1,06 Jan 2022
1,2,None,Director,Allan King,1,08 Dec 2016
2,3,None,CEO,Liza-Jayne Loch,1,25 Jan 2019
3,4,None,Director,Douglas Pye,1,01 Jun 2021
4,5,None,Senior Consultant,Amanda Parker,0,29 Sep 2016
...,...,...,...,...,...,...
1256,1257,None,Principal,Ken Macpherson,0,20 Jun 2018
1257,1258,None,Principal,Robert Borbidge AO,0,20 Jun 2018
1258,1259,None,Director,JoAnna Ferrari,0,29 Sep 2016
1259,1260,None,,Stephen Woodhill,1,29 Sep 2016


In [5]:
pd.read_sql_query("select * from lobbyist_nsw_client", cnx)

,id,lobbyist_nsw_id,name,abn,active,foreign_principal,countries,date_added
0,1,1,AUSTRALIA CHINA BUSINESS COUNCIL,57075909625,1,0,,06 Jan 2022
1,2,1,AUSTRALIAN FUNERAL DIRECTORS ASSOCIATION LTD,33007331580,1,0,,06 Jan 2022
2,3,1,INVOCARE LIMITED,42096437393,1,0,,06 Jan 2022
3,4,1,PARED LTD,40002343615,1,0,,06 Jan 2022
4,5,2,Genesis Care Finance Pty ltd,97137193125,1,0,,08 Dec 2016
...,...,...,...,...,...,...,...,...
4008,4009,345,PHILIP MORRIS (AUSTRALIA) LIMITED,68004316901,1,0,,08 Jan 2020
4009,4010,345,RED MEAT ADVISORY COUNCIL LIMITED,44083193891,0,0,,12 Feb 2020
4010,4011,345,RURALCO HOLDINGS LIMITED,40009660879,0,1,,30 Apr 2018
4011,4012,345,TERUMO AUSTRALIA PTY LIMITED,34158693595,0,1,,30 Apr 2018


In [6]:
pd.read_sql_query("select * from lobbyist_nsw_owner", cnx)

,id,lobbyist_nsw_id,name,active
0,1,None,Alistair Nicholas,1
1,2,None,Xiao Qi Bao,1
2,3,None,Allan King,1
3,4,None,Karen King,0
4,5,None,Liza-Jayne Loch,1
...,...,...,...,...
974,975,None,Martine Anne Newman Smail Woodhill,0
975,976,None,Martine Woodhill,0
976,977,None,Stephen Mark Arthur Woodhill,1
977,978,None,Martine Woodhill,1


In [7]:
pd.read_sql_query("select * from lobbyist_qld", cnx)

,id,name,abn,trading_name,last_updated
0,1,BBS Communications Group Pty Ltd,34010899779,BBS Communications Group Pty Ltd,12/07/2022
1,2,Candra Consulting Pty Ltd,26116275382,Candra Business Advisers,18/07/2022
2,3,Christopher Cleland Schacht,19927269400,Chris Schacht,19/10/2021
3,4,The Trustee for Consultum Trust,32844592574,Staerk Government and Media,01/08/2022
4,5,Essential Media Communications Pty Ltd,48074671462,Essential Media Communications,11/08/2022
...,...,...,...,...,...
125,126,The Civic Partnership Pty Ltd,71652574171,The Civic Partnership,03/08/2022
126,127,Advico Strategies Pty Ltd,52658982471,advico,05/07/2022
127,128,GR Connect Pty Ltd,40657487146,GR Connect Pty Ltd,01/07/2022
128,129,Test Only Pty Ltd,12345678910,Test Only,25/07/2022


In [10]:
pd.read_sql_query("select * from lobbyist_qld_employee", cnx)

,id,lobbyist_qld_id,name,position,former_senior_gov_rep,cessation_date,associations
0,1,1,Sara Benallack,Account Manager,0,,
1,2,1,Gina Calder,Account Leader,0,,
2,3,1,Louise Cruice,Group Client Manager,0,,
3,4,1,Verity Dwan,Senior Consultant,0,,
4,5,1,Amanda Firth,Group Client Manager,0,,
...,...,...,...,...,...,...,...
318,319,125,Eamonn Fitzpatrick,Managing Director,1,09/07/2013,
319,320,127,Nino Lalic,Director,0,,
320,321,128,Lyndal Maloney,Director,0,,
321,322,129,Test1 Only,Administrator,0,,


In [12]:
pd.read_sql_query("select * from lobbyist_qld_client", cnx)

,id,lobbyist_qld_id,name,paid_services_provided,client_added,made_previous
0,1,1,Avanti Healthy Living,1,15/06/2022,
1,2,1,AVID Property Group,1,02/08/2016,
2,3,1,Cystic Fibrosis Qld Pty Ltd,1,22/01/2020,
3,4,1,GCB Constructions Pty Ltd,1,17/12/2020,
4,5,1,Geon Property,1,05/03/2018,
...,...,...,...,...,...,...
1085,1086,126,Vicinity Centres PM Pty Ltd,0,21/04/2022,12/08/2022
1086,1087,129,Test Client 2 Only,1,20/07/2022,
1087,1088,129,Test Client Only,0,20/07/2022,20/07/2022
1088,1089,130,Deliveroo Australia Pty Ltd,1,19/07/2022,


In [14]:
pd.read_sql_query("select * from lobbyist_qld_owner", cnx)

,id,lobbyist_qld_id,name
0,1,1,Jane Edwards
1,2,1,Mrs Lisa Nixon
2,3,2,"Milin, Amanda"
3,4,2,Mr David Meale
4,5,3,The Honourable Chris Schacht
...,...,...,...
208,209,126,Mrs Rora Furman
209,210,126,The Civic Group Holdings Pty Ltd
210,211,127,Advico Strategies Pty Ltd
211,212,128,Mrs Lyndal Anne Maloney


In [4]:
from src.scrappers.LobbyistNswScrapper import *
import json
scrapper = LobbyistNswScrapper()
lobbyists = scrapper.get_lobbyists()

scrapper.populate_details(lobbyists[0])

for client in lobbyists[0].clients:
    print(f"{json.dumps(client.__dict__)}")


scrapper.close()

In [54]:
import json
for client in lobbyists[0].clients:
    print(f"{json.dumps(client.__dict__)}")

## Using Google to Search LinkedIn

In [15]:
#!pip install requests

In [20]:

import requests as re


from src.secrets import secrets as sc
import json

In [56]:
def custom_search(search_term, start = 1):
    custom_search_url = f"https://www.googleapis.com/customsearch/v1?key={sc.google_customsearch_api_key}&cx={sc.google_programmable_search_engine_id}&q={search_term}&start={start}"
    result = re.get(custom_search_url)
    return result.json()

def print_results(results):
    for item in results['items']:
        print(item['title'])
        print(item['link'])

This is what my profile link looks like:
https://www.linkedin.com/in/michael-gordon-b6698051/

In [73]:
mgordon_result = custom_search('"Michael Gordon" senior software LinkedIn', 21)
print_results(mgordon_result)

Sviatoslav Didukh - Associate Director, Experience Design - EPAM ...
https://ua.linkedin.com/in/sviatoslavdidukh
Michael Gordon | Harrison Street
https://www.harrisonst.com/leadership/michael-gordon/
Michael Gordon - Business Consultant - Self Employed | LinkedIn
https://tt.linkedin.com/in/michael-gordon-3b4a4723
Our Team - Group Gordon
https://groupgordon.com/people/
100+ perfiles de «Michael Gordon» | LinkedIn
https://bo.linkedin.com/pub/dir/Michael/Gordon/ca-0-Canada
Team | Angelo Gordon
https://www.angelogordon.com/team/
Michael Gordon - New Ireland Assurance - LinkedIn
https://ie.linkedin.com/in/michaelgordonmjg
NJDEP New Jersey Department of Environmental Protection
https://www.nj.gov/dep/easyaccess/commissoffice.htm
Amrita Samantasinghar - Software Engineer, Sr I - LinkedIn
https://in.linkedin.com/in/amrita-samantasinghar
Michael Gordon - Tshwane University of Technology - LinkedIn
https://za.linkedin.com/in/michael-gordon-b0831521


In [67]:
namin_result = custom_search('"Naeer Amin" LinkedIn')
print_results(namin_result)

Alyssa Clark - Administrative Coordinator - UPASS, HELPS - LinkedIn
https://au.linkedin.com/in/alyssa-clark-428588192
Jung Gyu Park - Software Engineer - SafetyCulture | LinkedIn
https://au.linkedin.com/in/jung-gyu-park-5b01a21b
Ciaran Cullen - Head of Talent - SafetyCulture | LinkedIn
https://au.linkedin.com/in/ciarancullen
Shuvra Sikder - Camden Council NSW Australia - LinkedIn
https://au.linkedin.com/in/shuvra-sikder-722126196
Anthony Muscat - Senior Frontend Developer - Pearson | LinkedIn
https://au.linkedin.com/in/anthony-muscat-b829676


In [68]:
schemweno_result = custom_search('"Sharon Chemweno" LinkedIn')
print_results(schemweno_result)

Sharon Chemweno - Ben-Gurion University of the Negev - LinkedIn
https://ke.linkedin.com/in/sharon-chemweno-b21b8517a
Ruth Cherogony - Project Intern - LinkedIn
https://il.linkedin.com/in/ruth-cherogony-749692234
100+ perfiles de «Chemweno» | LinkedIn
https://bo.linkedin.com/pub/dir/+/Chemweno/ke-0-Kenya
Jacob Maundu - Resolution Insurance Kenya - LinkedIn
https://ke.linkedin.com/in/jacob-maundu-5433a0173
Tania Acuna - Ben Gurion University of the Negev - LinkedIn
https://il.linkedin.com/in/tania-acuna-9a51a7b3
Beatrice Wambui - Case Manager - RefugePoint | LinkedIn
https://www.linkedin.com/in/beatrice-wambui-844855ba
Felecien NDAYISHIMIYE - WUA Technician - LinkedIn Rwanda
https://rw.linkedin.com/in/felecien-ndayishimiye-4a687a206
Jason Dorey - Director of Academics - CanGlory Education | LinkedIn
https://ca.linkedin.com/in/jason-dorey-b10308108
Pauline Ang'ang'o - Managing Editor - Genesis Tech | LinkedIn
https://hu.linkedin.com/in/pauline-ang-ang-o-194b55128
Mayumi Evangelista - Imme

In [69]:
gbohorquez_result = custom_search('"Gerardo Bohorquez Restrepo" LinkedIn')
print_results(gbohorquez_result)

Gerardo Restrepo - Senior Account Executive - LinkedIn
https://www.linkedin.com/in/gerardo-restrepo-7239757
Gerardo Restrepo - Professeur - Université de Sherbrooke | LinkedIn
https://ca.linkedin.com/in/gerardo-restrepo-675aa236
Gerardo Bohorquez - Colombia | Perfil profesional | LinkedIn
https://co.linkedin.com/in/gerardo-bohorquez-a7676290
40+ perfiles de «Bohorquez» | LinkedIn
https://bo.linkedin.com/pub/dir/+/Bohorquez/au-0-Australia
F5 group F5 group - Marketing Manager - F5 group | LinkedIn
https://co.linkedin.com/in/f5-group-f5-group-648a591b
Passiona Cottee - A/Director - LinkedIn
https://au.linkedin.com/in/passiona
Understanding Hierarchies using Dendrograms | by Yash Gupta ...
https://medium.com/dssimplified/understanding-hierarchies-using-dendrograms-e3aef7ac5ea4
